In [ ]:
import pandas as pd
import numpy as np
from pandas import Series
import pickle

In [ ]:
# combine app data and extract youtube and instagram data

In [ ]:
app1 = pd.read_csv('[APP] Refine_Usage_Sum_Time(60~63).csv')
app2 = pd.read_csv('[APP] Refine_Usage_Sum_Time(64~67).csv')
app3 = pd.read_csv('[APP] Refine_Usage_Sum_Time(68~72).csv')
app4 = pd.read_csv('[APP] Refine_Usage_Sum_Time(73~76).csv')

In [ ]:
print('app1 : ',app1.shape)
print('app2 : ',app2.shape)
print('app3 : ',app3.shape)
print('app4 : ',app4.shape)

In [ ]:
app1.head()

In [ ]:
app2.head()

In [ ]:
app3.head()

In [ ]:
app4.head()

In [ ]:
app = pd.concat([app1, app2, app3, app4], ignore_index = True)

In [ ]:
app_youtube = app[app['app_name'] == 'YouTube']
app_instagram = app[app['app_name'] == 'Instagram']

In [ ]:
print(app_youtube.shape)
app_youtube.head()

In [ ]:
print(app_instagram.shape)
app_instagram.head()

In [ ]:
youtube_id = []
instagram_id = []
for i in app_youtube['panel_id'].unique() : 
    youtube_id.append(i)
for i in app_instagram['panel_id'].unique() : 
    instagram_id.append(i)

In [ ]:
len(youtube_id)

In [ ]:
id_for_both = set().union(youtube_id, instagram_id)

In [ ]:
len(id_for_both)

In [ ]:
#Find the panel data of youtube and instagram user

In [ ]:
panel = pd.read_csv('[PPDB] data.csv', index_col = 'id')

In [ ]:
print(panel.shape)
panel.head()

In [ ]:
panel_id = set(panel.index)

In [ ]:
panel_and_app_id = panel_id & id_for_both

In [ ]:
panel = panel[panel.index.isin(panel_and_app_id)]

In [ ]:
#아이디별로 사용하는 시간
youtubetime = app_youtube.groupby(['panel_id']).mean()
youtubetime = youtubetime[youtubetime.index.isin(panel_and_app_id)]
print(len(youtubetime))


instagramtime = app_instagram.groupby(['panel_id']).mean()
instagramtime = instagramtime[instagramtime.index.isin(panel_and_app_id)]
print(len(instagramtime))

In [ ]:
youtubetime = youtubetime[['total_time_stamp']]
instagramtime = instagramtime[['total_time_stamp']]

In [ ]:
youtubetime[0:10]

In [ ]:
panel = panel.join(youtubetime)
panel.rename(columns = {'total_time_stamp' : 'youtubetime'},inplace = True)

In [ ]:
panel = panel.join(instagramtime)
panel.rename(columns = {'total_time_stamp' : 'instagramtime'}, inplace = True)

In [ ]:
# because of the data limitation, save the rearranged data set as a pickle file
pickle_out = open('app_and_panel.pickle', 'wb')
pickle.dump(panel, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open('app_and_panel.pickle', 'rb')
panel = pickle.load(pickle_in)

In [ ]:
panel

In [ ]:
# combine payment data and extract row with app_and_panel IDs and selected columns

In [ ]:
pay1 = pd.read_csv('[PAYMENT] PAYMENT_201811.csv')
pay2 = pd.read_csv('[PAYMENT] PAYMENT_201812.csv')
pay3 = pd.read_csv('[PAYMENT] PAYMENT_201901.csv')
pay4 = pd.read_csv('[PAYMENT] PAYMENT_201902.csv')

In [ ]:
pay = pd.concat([pay1, pay2, pay3, pay4])

In [ ]:
print(pay.shape)
pay.head()

In [ ]:
pay = pay[['PANEL_ID', 'age', 'Area_Code1', 'Area_Code2', 'CATEGORY_CODE', 'CATEGORY_GROUP_CODE', 'CARD_PAYMENT_TYPE', 'APPROVAL_REAL_PRICE']]

In [ ]:
pickle_out = open('pay.pickle', 'wb')
pickle.dump(pay, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open('pay.pickle', 'rb')
pay = pickle.load(pickle_in)

In [ ]:
# make copy of the pay dataframe
payment = pay.copy()
payment.CARD_PAYMENT_TYPE = payment.CARD_PAYMENT_TYPE.str.rstrip()

In [ ]:
payment.head()

In [ ]:
payment['CARD_PAYMENT_TYPE'].replace('PBK', 0, inplace = True)
payment['CARD_PAYMENT_TYPE'].replace('PCK', 0, inplace = True)
payment['CARD_PAYMENT_TYPE'].replace('PCT', 0, inplace = True)
payment['CARD_PAYMENT_TYPE'].replace('SP', 0, inplace = True)
payment['CARD_PAYMENT_TYPE'].replace('PA', 1, inplace = True)

In [ ]:
payment.head()

In [ ]:
payment.dtypes

In [ ]:
payment['ratio_pa'] = pd.to_numeric(payment["CARD_PAYMENT_TYPE"])

In [ ]:
pay_id = set(payment['PANEL_ID'].unique())

In [ ]:
total_id = pay_id & set(panel.index)

In [ ]:
len(total_id)

In [ ]:
panel=panel[panel.index.isin(total_id)]

In [ ]:
print(panel.shape)
panel.head()

In [ ]:
ratio = payment.groupby('PANEL_ID').mean()
ratio = ratio[['ratio_pa']]

In [ ]:
panel = panel.join(ratio)

In [ ]:
print(panel.shape)
panel.head()

In [ ]:
payX=pay[['PANEL_ID', 'age','Area_Code1','Area_Code2']]

In [ ]:
payX.Area_Code1 = payX.Area_Code1.str.strip()
payX["Area_Code1"] = pd.to_numeric(payX['Area_Code1'], downcast='float')

In [ ]:
payX.Area_Code2 = payX.Area_Code2.str.strip()
payX["Area_Code2"] = pd.to_numeric(payX['Area_Code2'], downcast='float')

In [ ]:
payX = payX.groupby(['PANEL_ID']).mean()

In [ ]:
payX['age'] = np.ceil(payX['age'])

In [ ]:
panel.join(payX)

In [ ]:
pickle_out = open('dataframe_bigbrain.pickle', 'wb')
pickle.dump(panel, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open('dataframe_bigbrain.pickle', 'rb')
panel = pickle.load(pickle_in)

In [ ]:
pay.head()

In [ ]:
# divide the payment into Categories and select categories with significant number of data

In [ ]:
CVs = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==101)|(pay["CATEGORY_CODE"]==199)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
CVs.rename(columns = {'APPROVAL_REAL_PRICE' : 'CVs'},inplace = True)
len(CVs)

In [ ]:
emart = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==201)|(pay["CATEGORY_CODE"]==202)|(pay["CATEGORY_CODE"]==299)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
emart.rename(columns = {'APPROVAL_REAL_PRICE' : 'emart'},inplace = True)
len(emart)

In [ ]:
cafe = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==301)|(pay["CATEGORY_CODE"]==399)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
cafe.rename(columns = {'APPROVAL_REAL_PRICE' : 'cafe'},inplace = True)
len(cafe)

In [ ]:
food = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==401)|(pay["CATEGORY_CODE"]==499)|(pay["CATEGORY_CODE"]==402)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
food.rename(columns = {'APPROVAL_REAL_PRICE' : 'food'},inplace = True)
len(food)

In [ ]:
on_shop = pd.DataFrame(pay[pay["CATEGORY_CODE"]==501].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
on_shop.rename(columns = {'APPROVAL_REAL_PRICE' : 'on_shop'},inplace = True)
len(on_shop)

In [ ]:
Complex = pd.DataFrame(pay[pay["CATEGORY_CODE"]==502].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
Complex.rename(columns = {'APPROVAL_REAL_PRICE' : 'Complex'},inplace = True)
len(Complex)

In [ ]:
furniture = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==504)|(pay["CATEGORY_CODE"]==505)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
furniture.rename(columns = {'APPROVAL_REAL_PRICE' : 'furniture'},inplace = True)
len(furniture)

In [ ]:
fashion = pd.DataFrame(pay[pay["CATEGORY_CODE"]==601].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
fashion.rename(columns = {'APPROVAL_REAL_PRICE' : 'fashion'},inplace = True)
len(fashion)

In [ ]:
beauty = pd.DataFrame(pay[pay["CATEGORY_CODE"]==602].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
beauty.rename(columns = {'APPROVAL_REAL_PRICE' : 'beauty'},inplace = True)
len(beauty)

In [ ]:
subway = pd.DataFrame(pay[pay["CATEGORY_CODE"]==701].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
subway.rename(columns = {'APPROVAL_REAL_PRICE' : 'subway'},inplace = True)
len(subway)

In [ ]:
taxi = pd.DataFrame(pay[pay["CATEGORY_CODE"]==702].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
taxi.rename(columns = {'APPROVAL_REAL_PRICE' : 'taxi'},inplace = True)
len(taxi)

In [ ]:
car_ma = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==703)|(pay["CATEGORY_CODE"]==704)|(pay["CATEGORY_CODE"]==705)|(pay["CATEGORY_CODE"]==706)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
car_ma.rename(columns = {'APPROVAL_REAL_PRICE' : 'car_ma'},inplace = True)
len(car_ma)

In [ ]:
internet = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==801)|(pay["CATEGORY_CODE"]==899)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
internet.rename(columns = {'APPROVAL_REAL_PRICE' : 'internet'},inplace = True)
len(internet)

In [ ]:
book = pd.DataFrame(pay[pay["CATEGORY_CODE"]==1001].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
book.rename(columns = {'APPROVAL_REAL_PRICE' : 'book'},inplace = True)
len(book)

In [ ]:
hospital = pd.DataFrame(pay[(pay["CATEGORY_CODE"]==1102)|(pay["CATEGORY_CODE"]==1103)|(pay["CATEGORY_CODE"]==1101)].groupby(['PANEL_ID']).sum().APPROVAL_REAL_PRICE)
hospital.rename(columns = {'APPROVAL_REAL_PRICE' : 'hospital'},inplace = True)
len(hospital)

In [ ]:
price = CVs.join(emart).join(cafe).join(food).join(on_shop).join(Complex).join(furniture).join(fashion).join(beauty).join(subway).join(taxi).join(car_ma).join(internet).join(book).join(hospital)

In [ ]:
print(price.shape)
price.head()

In [ ]:
pickle_out = open('price.pickle', 'wb')
pickle.dump(price, pickle_out)
pickle_out.close()